In [1]:
import os

In [2]:
%pwd

'd:\\ml projects\\End-To-End-MLOPs-Credit_Card_Fraud_Detection-main\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ml projects\\End-To-End-MLOPs-Credit_Card_Fraud_Detection-main'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    """
    A class for holding an data path
    """
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    
    


In [7]:
from Credit_Card_Fraud_Detection.constants import *
from Credit_Card_Fraud_Detection.utils.common import read_yaml,create_directories

In [8]:
print(CONFIG_FILE_PATH)

config\config.yaml


In [9]:
print(PARAMS_FILE_PATH)

params.yaml


In [10]:
print(SCHEMA_FILE_PATH)

schema.yaml


In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        print(self.config)
        print(self.config.artifacts_root)
        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])   
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir,
        )
        
        return data_ingestion_config

 

In [20]:
import os
import urllib.request as request
import zipfile
from Credit_Card_Fraud_Detection import logger
from Credit_Card_Fraud_Detection.utils.common import get_size

In [21]:
class DataIngestionStrategy:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
        print("="*50)
        print(self.config.local_data_file)
        print("="*50)
        
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with the following \n {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  
        


In [23]:
# Pipelines for the DataIngestionStrategy
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestionStrategy(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-12-27 23:29:23,951: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-27 23:29:23,958: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-27 23:29:23,963: INFO: common: yaml file: schema.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/Rupesh-132/End-To-End-MLOPs-Credit_Card_Fraud_Detection/raw/main/data/credit_card_dataset.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_dir': 'artifacts/data_ingestion/creditcard.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/creditcard.csv'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_data_path': 'artifacts/data_transformation/train.csv',